In [36]:
import pandas as pd
from pandas import read_csv, DataFrame

df1 = read_csv('ChEC-LEQ_rated_20200511.csv', index_col=None)
df2 = read_csv('LEQ_EmoGrow_JC_06152020_rated.csv', index_col=None)

df3 = pd.concat([df1, df2])

df3.to_csv('combined.csv')

In [37]:
data = read_csv('combined.csv', index_col=None)

In [38]:
prefixes = ['ser_acc','wit_ser_acc','nat_dis','oth_ill','death_oth','death_pet','med_pro','sep',
            'suicide','assault','threat','kidnap','an_att','wit_fam_vio','wit_fam_threat',
            'wit_oth_vio','yell','lack_care','new_sib','new_scho','new_home','change_fam','hobby',
            'new_friend','end_friend','finan','recog','oth_stress','oth_pos']
neg_pref = ['ser_acc','wit_ser_acc','nat_dis','oth_ill','death_oth','death_pet','med_pro','sep',
            'suicide','assault','threat','kidnap','an_att','wit_fam_vio','wit_fam_threat',
            'wit_oth_vio','yell','lack_care','oth_stress']
pos_pref = ['hobby','new_friend','recog','oth_pos']
ambig_pref = ['new_sib','new_scho','new_home','change_fam','end_friend','finan']
psychosocial = ['wit_serious_accident', 'other_illnessinjury', 'death_person', 'death_pet', 
                'separation', 'suicide', 'threat_harm', 'wit_fam_violence', 'wit_fam_threat', 
                'wit_oth_violence', 'yell', 'lack_care', 'finance']
physical = ['serious_accident', 'nat_disaster', 'med_procedures', 'assault', 'animal_attack']


# subs = 

In [39]:
import numpy as np

data['LEQ_tot_neg'] = 0
data['LEQ_sum_sev_neg'] = 0
data['LEQ_tot_pos'] = 0
data['LEQ_sum_sev_pos'] = 0
data['LEQ_tot_endorsed'] = 0
data['LEQ_tot_psych'] = 0
# data['LEQ_sum_sev_psych'] = 0
data['LEQ_tot_phys'] = 0
# data['LEQ_sum_sev_phys'] = 0



# loop through and start making summary codes
for e in prefixes:
    data[e+'_num_neg_events'] = 0
    data[e+'_num_pos_events'] = 0
    data[e+'_tot_endorsed'] = 0
    data[e+'_num_psych_events'] = 0
    data[e+'_num_phys_events'] = 0
    data[e+'_num_neg_events'] = (data[e+'_imp1']<0)*1+(data[e+'_imp2']<0)*1+(data[e+'_imp3']<0)*1+(data[e+'_imp4']<0)*1
    data[e+'_num_pos_events'] = (data[e+'_imp1']>0)*1+(data[e+'_imp2']>0)*1+(data[e+'_imp3']>0)*1+(data[e+'_imp4']>0)*1
    data[e+'_tot_endorsed'] = (np.isfinite(data[e+'_imp1']))*1+(np.isfinite(data[e+'_imp2']))*1+(np.isfinite(data[e+'_imp3']))*1+(np.isfinite(data[e+'_imp4']))*1
    # type, 0=phys, 1 = psych
    data[e+'_num_psych_events'] = (data[e+'_type1']>0)*1+(data[e+'_type2']>0)*1+(data[e+'_type3']>0)*1+(data[e+'_type4']>0)*1
    data[e+'_num_phys_events'] = (data[e+'_type1']<1)*1+(data[e+'_type2']<1)*1+(data[e+'_type3']<1)*1+(data[e+'_type4']<1)*1
    
    
    negtemp = DataFrame(columns=['_imp1','_imp2','_imp3','_imp4'], index=data.index)
    postemp = DataFrame(columns=['_imp1','_imp2','_imp3','_imp4'], index=data.index)
    for i in ['_imp1','_imp2','_imp3','_imp4']:
        negtemp[i] = abs(data[data[e+i]<0][e+i])
        postemp[i] = data[data[e+i]>0][e+i]
    negtemp.fillna(0,inplace=True)
    postemp.fillna(0, inplace=True)

    data[e+'_sum_sev_neg'] = negtemp.sum(axis=1)
    data[e+'_sum_sev_pos'] = postemp.sum(axis=1)
 

    data['LEQ_tot_neg'] = data['LEQ_tot_neg'] + data[e+'_num_neg_events']
    data['LEQ_sum_sev_neg'] = data['LEQ_sum_sev_neg'] + data[e+'_sum_sev_neg']
    data['LEQ_tot_pos'] = data['LEQ_tot_pos'] + data[e+'_num_pos_events']
    data['LEQ_sum_sev_pos'] = data['LEQ_sum_sev_pos'] + data[e+'_sum_sev_pos']
    data['LEQ_tot_endorsed'] = data['LEQ_tot_endorsed'] + data[e+'_tot_endorsed'] 
    data['LEQ_tot_psych'] = data['LEQ_tot_psych'] + data[e+'_num_psych_events']
    data['LEQ_tot_phys'] = data['LEQ_tot_phys'] + data[e+'_num_phys_events']
    
 
    
                                                                                                                                           

In [40]:
data.to_csv('LEQ_Combined_all_scored.csv')

In [1]:
import pandas as pd
from pandas import read_csv, DataFrame
import matplotlib.pyplot as plt
import seaborn as sns

scored_df = read_csv('LEQ_Combined_all_scored.csv', index_col=None)
# drop subjects that don't have DWI by index
DWI_sample = scored_df.drop([2, 3, 4, 5, 9, 10, 13, 14, 18, 19, 20, 21, 22, 24, 28, 
                29, 30, 33, 35, 37, 38, 39, 41, 42, 47, 48, 52, 54, 55,
               56, 57, 58, 67, 77, 90, 100, 109, 123, 125, 132, 133, 
                134, 135, 136, 138, 145, 148, 153, 155, 158, 161,
               163, 171, 172, 178, 185, 188, 189, 190, 191, 192, 193,
               194, 195, 196, 198, 199, 200, 201, 202,203, 204, 205, 
               206, 207, 208, 209])

# index = DWI_sample.index
# sample = len(index)
# print(sample)

# full_index = scored_df.index
# full_sample = len(full_index)
# print(full_sample)

In [ ]:
#categorical histogram

In [2]:
# #count histograms

sns.distplot(scored_df['LEQ_tot_endorsed'],kde=False, color='purple')
plt.suptitle('Total Event Endorsement (DWI Sample)')
plt.xlabel('events endorsed')
plt.ylabel('subject frequency')
plt.savefig('plots/LEQ_tot_endorsed.png')
plt.clf()

sns.distplot(DWI_sample['LEQ_tot_endorsed'],kde=False, color='purple')
plt.suptitle('Total Event Endorsement (Full Sample)')
plt.xlabel('events endorsed')
plt.ylabel('subject frequency')
plt.savefig('plots/LEQ_tot_endorsed_fullSample.png')
plt.clf()

sns.distplot(DWI_sample['LEQ_tot_psych'], kde=False, color='#00B9DE')
plt.suptitle('Psychosocial Event Endorsement')
plt.xlabel('events endorsed')
plt.ylabel('subject frequency')
plt.savefig('plots/LEQ_tot_psych.png')
plt.clf()

sns.distplot(DWI_sample['LEQ_tot_phys'], kde=False, color='#0AE1A0')
plt.suptitle('Physical Event Endorsement')
plt.xlabel('events endorsed')
plt.ylabel('subject frequency')
plt.savefig('plots/LEQ_tot_phys.png')
plt.clf()


# sns.distplot(DWI_sample['LEQ_tot_neg'],kde=False, color='purple')
# plt.suptitle('Negative Event Endorsement')
# plt.xlabel('events endorsed')
# plt.ylabel('count')
# plt.savefig('plots/LEQ_tot_neg.png')
# plt.clf()

# sns.distplot(DWI_sample['LEQ_tot_pos'], kde=False, color='purple')
# plt.suptitle('Positive Event Endorsement')
# plt.xlabel('events endorsed')
# plt.ylabel('count')
# plt.savefig('plots/LEQ_tot_pos.png')
# plt.clf()

<Figure size 432x288 with 0 Axes>

In [46]:
#scored histograms
#DTI sample
sns.distplot(DWI_sample['LEQ_sum_sev_neg'], kde=False, color='red')
plt.suptitle('Sum Severity Negative Events (DWI Sample)')
plt.xlabel('sum severity')
plt.ylabel('subject frequency')
plt.savefig('plots/sum_sev_neg.png')
plt.clf()

sns.distplot(DWI_sample['LEQ_sum_sev_pos'], kde=False, color='blue')
plt.suptitle('Sum Severity Positive Events (DWI Sample)')
plt.xlabel('sum severity')
plt.ylabel('subject_frequency')
plt.savefig('plots/sum_sev_pos.png')
plt.clf()

#Full sample
sns.distplot(scored_df['LEQ_sum_sev_neg'], kde=False, color='red')
plt.suptitle('Sum Severity Negative Events (Full Sample)')
plt.xlabel('sum severity')
plt.ylabel('subject frequency')
plt.savefig('plots/sum_sev_neg_fullSample.png')
plt.clf()

sns.distplot(scored_df['LEQ_sum_sev_pos'], kde=False, color='blue')
plt.suptitle('Sum Severity Positive Events (Full Sample)')
plt.xlabel('sum severity')
plt.ylabel('subject_frequency')
plt.savefig('plots/sum_sev_pos_fullSample.png')
plt.clf()


<Figure size 432x288 with 0 Axes>